<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [ ]:
!pip install tensorflow-addons==0.10.0
!pip install oyaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scikit-image tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/LynnHo/CycleGAN-Tensorflow-2.git

Cloning into 'CycleGAN-Tensorflow-2'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 203 (delta 4), reused 4 (delta 2), pack-reused 194
Receiving objects: 100% (203/203), 2.63 MiB | 1.93 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [ ]:
import os
os.chdir('CycleGAN-Tensorflow-2/')

In [ ]:
!ls

data.py   download_dataset.sh  LICENSE	  pics	 README.md  tf2gan  train.py
datasets  imlib		       module.py  pylib  test.py    tf2lib


# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [ ]:
!bash ./download_dataset.sh horse2zebra

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
!bash ./scripts/download_cyclegan_model.sh horse2zebra

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [ ]:
!python train.py --dataset horse2zebra --epochs 20

Streaming output truncated to the last 5000 lines.
Inner Epoch Loop: 100% 1333/1334 [16:57<00:00,  1.31it/s]
Inner Epoch Loop: 100% 1334/1334 [16:58<00:00,  1.31it/s]
Epoch Loop:  60% 12/20 [3:24:53<2:16:05, 1020.71s/it]
Inner Epoch Loop:   0% 0/1334 [00:00<?, ?it/s]
Inner Epoch Loop:   0% 1/1334 [00:00<18:39,  1.19it/s]
Inner Epoch Loop:   0% 2/1334 [00:01<17:36,  1.26it/s]
Inner Epoch Loop:   0% 3/1334 [00:02<17:14,  1.29it/s]
Inner Epoch Loop:   0% 4/1334 [00:03<17:04,  1.30it/s]
Inner Epoch Loop:   0% 5/1334 [00:03<16:59,  1.30it/s]
Inner Epoch Loop:   0% 6/1334 [00:04<16:54,  1.31it/s]
Inner Epoch Loop:   1% 7/1334 [00:05<16:53,  1.31it/s]
Inner Epoch Loop:   1% 8/1334 [00:06<16:49,  1.31it/s]
Inner Epoch Loop:   1% 9/1334 [00:06<16:46,  1.32it/s]
Inner Epoch Loop:   1% 10/1334 [00:07<16:46,  1.32it/s]
Inner Epoch Loop:   1% 11/1334 [00:08<16:42,  1.32it/s]
Inner Epoch Loop:   1% 12/1334 [00:09<16:43,  1.32it/s]
Inner Epoch Loop:   1% 13/1334 [00:09<16:42,  1.32it/s]
Inner Epoch L

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
CUDA_VISIBLE_DEVICES=0 python main.py --dataset_dir=horse2zebra --phase=test --which_direction=AtoB

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)